## 7. Atributos

[Playlist de Ciencia de Datos en castellano](https://www.youtube.com/watch?v=w97CsaLuEvI&list=PLLBUgWXdTBDg1Qgmwt4jKtVn9BWh5-zgy)
[![Ciencia de Datos en Python](https://img1.wsimg.com/isteam/ip/aab852a2-7b1f-49c0-92af-9206f2ec6a75/8-0001.png/:/rs=w:1160,h:653)](https://www.youtube.com/watch?v=w97CsaLuEvI&list=PLLBUgWXdTBDg1Qgmwt4jKtVn9BWh5-zgy "Python Data Science")

**La clasificación** predice *etiquetas discretas (resultados)* como `sí` /` no`, `Verdadero` /` Falso`, o cualquier tipo de variable discreta, como una letra mediante de reconocimiento de texto. Un ejemplo de clasificación es sugerir una película que querrás ver (etiqueta) en base al historial de visualización (o atributo(s)). **La Regresión**, a diferencia de la clasificación, da resultados continuos, como un número real dentro de un rango. Un ejemplo de regresión es construir una correlación de la temperatura de una olla con agua (etiqueta) basada en el tiempo que se ha estado calentando (atributo). Los valores de temperatura son continuos, mientras que la siguiente película es una de las muchas opciones discretas.

![list](https://apmonitor.com/che263/uploads/Begin_Python/list.png)

Los atributos son variables de entrada para modelos de regresión o clasificación. **Los atributos son entradas** y **las etiquetas son los resultados medidos**. A continuación se muestra una tabla de términos con terminología de aprendizaje automático (machine learning), optimización, "GEKKO" y una breve descripción.


| **Machine Learning** | **Optimización** | **Estimación en Gekko** | **Descripción** |
| ----------- | ----------- | ----------- | ----------- |
| Pérdida | Función objetivo | `m.Minimize()` | La representación matemática de la diferencia entre los valores medidos y los valores predichos |
| Ponderaciones | Parámetros ajustables | Valores fijos (`m.FV()`) con `STATUS=1` | Valores ajustables para minimizar la función objetivo |
| Etiqueta | Resultado medido | Variable controlada (`m.CV()`) con `FSTATUS=1` | Mediciones u observaciones de las variables predichas |
| Atributo | Entrada medida | Parámetro (`m.Param()`) | Mediciones de entrada que se usan para predecir las etiquetas (resultados) a la salida |
| Entrenar | Optimizar | Resolver (`m.solve()`) | Ajustar los parámetros ajustables (ponderaciones) para minimizar la función objetivo (pérdida) |
| Testear | Evaluar | Resolver `STATUS=0` para `m.FV()` | Predecir las etiquetas con el modelo ajustado para evaluar el desempeño del clasificador o regresor | 
| Regresor o Clasificador | Modelo | `m = GEKKO()` | Ecuaciones matemáticas y parámetros que utilizan los atributos de entrada y devuelven como resultado las etiquetas de salida |

![idea](https://apmonitor.com/che263/uploads/Begin_Python/idea.png)

La selección y creación de atributos es un paso importante en machine learning. Un número muy grande de atributos podría provocar un incremento en la probabilidad de que el regresor o clasificador tenga errores en las predicciones. Con muchos atributos, una entrada podría ser un valor "malo" y causar una predicción errónea. Además, mientras más atributos, más tiempo es necesario para limpiar los datos, entrenar los modelos y predecir resultados. A continuación se mostrarán métodos para seleccionar los mejores atributos para clasificación y regresión.

![analyze](https://apmonitor.com/che263/uploads/Begin_Python/analyze.png)

### Identificar Atributos y Etiquetas

El primer paso para construir un regresor o clasificador es determinar qué medidas (atributos de entrada y etiquetas de salida) están disponibles. Puedes seleccionar las columnas de datos como atributos o generar [atributos derivados](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219) con una librería como [`Featuretools`](https://towardsdatascience.com/why-automated-feature-engineering-will-change-the-way-you-do-machine-learning-5c15bf188b96).

Es posible que gustes utilizar los datos del mercado de valores para obtener un indicador de cuándo comprar (`1`) o cuándo vender (`-1`). Este indicador es una etiqueta. Importa los datos de las acciones diarias de Google durante 23 días.


In [ ]:
import pandas as pd
import numpy as np
url = 'http://apmonitor.com/che263/uploads/Main/goog.csv'
data = pd.read_csv(url)
data = data.drop(columns=['Adj Close'])
data.head()

Los atributos pueden ser cualquiera de las categorías que puedan ser útiles para predecir un cambio futuro en el precio de las acciones. La columna `Open`, la diferencia entre las columnas `High` y `Low` (`Volatility`), la diferencia entre las columnas `Close` y `Open` (`Change`) y la columna `Volume` son ejemplos de atributos. Con `.diff ()` se calcula la diferencia, y con `.fillna (0)` se reemplaza cualquier valor `NaN` por cero. Agrega cualquier otro atributo adicional que te gustaría considerar.

In [ ]:
features = ['Open','Volatility','Change','Volume']
data['Volatility'] = (data['High']-data['Low']).diff()
data['Change'] = (data['Close']-data['Open']).diff()
# Algún otro atributo?
data.head()

Una etiqueta (resultado) para la clasificación es el signo (`+` o `-`) del precio de cierre de un día para otro. Con `np.roll (, -1)` se desplazan todos los valores una casilla hacia arriba para indicar el cambio respecto al día siguiente en esa misma fila. Con `np.sign ()` se obtiene el signo de la diferencia como un indicador de compra o venta, y `.dropna ()` elimina la última fila que es `NaN`.

In [ ]:
data['Close_diff'] = np.roll(data['Close'].diff(),-1)
data=data.dropna()
label = ['Buy/Sell']
data['Buy/Sell'] = np.sign(data['Close_diff'])
data.head()

![power](https://apmonitor.com/che263/uploads/Begin_Python/power.png)

### Seleccionar los Mejores Atributos

Ahora que hemos creado varios atributos queremos seleccionar los mejores para predecir las etiquetas de salida. Hay varios métodos para evaluar cuántos atributos se necesitan (correlación) y cuáles son los mejores (selección). El primer paso es separar la entrada `X` y la salida` Y` mediante una escala de los datos (de `0` a` 1`).

In [ ]:
data[features+label].head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
s = MinMaxScaler()
ds = s.fit_transform(data[features+label])
ds = pd.DataFrame(ds,columns=data[features+label].columns)
X = ds[features]
y = ds[label]
ds.head()

![idea](https://apmonitor.com/che263/uploads/Begin_Python/idea.png)

#### Selección

Existen pruebas estadísticas que muestran cuáles atributos mantienen una fuerte relación con la etiqueta de salida. Una herramienta es el método de scikit-learn `SelectKBest` que tiene pruebas estadísticas asociadas. Este método utiliza una prueba $ \ chi ^ 2 $ para atributos no negativos y selecciona 10 de los mejores atributos para predecir la salida.

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
%matplotlib inline
bestfeatures = SelectKBest(score_func=chi2, k='all')
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
scores = pd.concat([dfcolumns,dfscores],axis=1)
scores.columns = ['Specs','Score']
scores.index = features
scores.plot(kind='bar')
plt.show()

![expert](https://apmonitor.com/che263/uploads/Begin_Python/expert.png)

En base a esta información, elimina cualquier atributo que tenga puntuación baja con `.remove ()`.

In [ ]:
# Elimine cualquier atributo con puntuación baja usando features.remove('')
print(features)

![idea](https://apmonitor.com/che263/uploads/Begin_Python/idea.png)

#### Importancia del Atributo

Existe un método que utiliza un clasificador en base a árboles (Tree Based Classifier) que califica a cada atributo. Una calificación alta significa más importancia y relevancia en la predicción de la variable de salida. Los resultados cambian con cada análisis, debido a la naturaleza estocástica del cálculo. No obstante, la Volatilidad (`Volatility`) es un factor que normalmente puntúa entre los más altos.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(n_estimators=100)
model.fit(X,np.ravel(y))

feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(4).plot(kind='bar')
plt.show()

![idea](https://apmonitor.com/che263/uploads/Begin_Python/idea.png)

#### Matriz de Correlación con Mapa de Calor

La correlación nos muestra qué tanto se relacionan unos atributos con otros. Un valor elevado, ya sea positivo o negativo, indica que las variables están relacionadas. Si se tienen variables correlacionadas entonces puede eliminarse una de ellas, pues ambas proveen información similar. Un mapa de calor es una cuadrícula visual simétrica de la matriz de correlación. La diagonal siempre toma el valor de 1, pues cualquier variable se relaciona perfectamente consigo misma. En base al mapa de calor, determina qué variables están más relacionadas y podrían eliminarse.

In [ ]:
import seaborn as sns
corrmat = ds.corr()
top_features = corrmat.index
plt.figure(figsize=(5,5))
sns.heatmap(ds[top_features].corr(),annot=True,cmap="RdYlGn") #
b, t = plt.ylim(); plt.ylim(b+0.5, t-0.5) # soluciona el problema en matplotlib 3.1.1
plt.show()

![expert](https://apmonitor.com/che263/uploads/Begin_Python/expert.png)

### Actividad con el TCLab

![connections](https://apmonitor.com/che263/uploads/Begin_Python/connections.png)

Considera cómo se podría determinar si el calentador del TCLab está encendido o apagado sin conocer el valor `Q1`. Probablemente mediríamos la temperatura `T1` y se observaría si la temperatura está subiendo o bajando. Sin embargo, simplemente observar la temperatura y la pendiente no es suficiente porque la temperatura continúa aumentando durante 10 a 20 segundos, después de que se apaga el calentador. Por tal motivo, debe calcular la segunda derivada de la temperatura para clasificar el estado del calentador como encendido o apagado. Una segunda derivada positiva es otra pista de que el calentador está encendido.

![temperature](https://apmonitor.com/che263/uploads/Begin_Python/temperature.png)

Ya sea que el calentador esté encendido o apagado (*resultado medido*), la temperatura y las derivadas son los *atributos*. Ejecuta el siguiente código para generar datos de temperatura con el calentador encendido al 100% o apagado al 0% en intervalos de 20 a 30 segundos durante 3 minutos.

In [ ]:
import tclab, time
import numpy as np
import pandas as pd
try:
    with tclab.TCLab() as lab:
        n = 180; t = np.linspace(0,n-1,n)        
        Q1 = np.zeros(n); T1 = np.zeros(n)
        Q2 = np.zeros(n); T2 = np.zeros(n)        
        Q1[20:41] = 100.0; Q1[60:91] = 100.0
        Q1[150:181] = 100.0; Q1[190:206] = 100.0
        Q1[220:251] = 100.0; Q1[260:291] = 100.0
        print('Tiempo Q1 Q2 T1   T2')
        for i in range(180):
            T1[i] = lab.T1; T2[i] = lab.T2
            lab.Q1(Q1[i])
            if i%10==0:
                print(int(t[i]),Q1[i],Q2[i],T1[i],T2[i])
            time.sleep(1)
    data = np.column_stack((t,Q1,Q2,T1,T2))
    data7 = pd.DataFrame(data,columns=['Tiempo','Q1','Q2','T1','T2'])
    data7.to_csv('07-tclab.csv',index=False)
except:
    print('Conecte el TCLab para generar nuevos datos')
    print('Importando datos en línea')
    url = 'http://apmonitor.com/do/uploads/Main/tclab_data5.txt'
    data7 = pd.read_csv(url)

### Crear Atributos

Crea tres atributos a partir de los datos incluyendo la temperatura y sus derivadas.

- Temperatura: $T_1$
- Primera derivada de la temperatura: $\frac{dT_1}{dt}$
- Segunda derivada de la temperatura: $\frac{d^2T_1}{dt^2}$

Añade las derivadas como columnas en `data7`.

### Escalar Datos

Escala `data7` de forma que resulten valores entre `0` y `1` utilizando `d7 = s.fit_transform(data7)`. No olvides volver a traducir los valores escalados a un `pandas` DataFrame.

### Calificar Atributos

Utiliza `SelectKBest` para determinar los mejores atributos.

### Correlacionar Atributos

Genera un mapa de calor para determinar la correlación entre los atributos. 